In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import gmaps.datasets
import os
import gmaps

# Import API key
from api_keys import g_key

#Imprort output from data acquisition and cleaning notebook
clean_msp_home_sales_df = pd.DataFrame(pd.read_csv("clean_msp_home_sales.csv"))
clean_msp_home_sales_df.set_index('sale_id', inplace=True)

#Extract a short df for testing API's etc
short_msp_home_sales_df = clean_msp_home_sales_df.iloc[0:5]

print(g_key)
short_msp_home_sales_df

In [ ]:
params = {
    "key": g_key,
}

for index, row in short_msp_home_sales_df.iterrows():
    # get address from df
    address = row["formatted_address"]
    address = address.replace("#", "%23")

    # change location each iteration while leaving original params in place
    params["address"] = f"{address}"
    
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={g_key}'

    #Base url and request
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        short_msp_home_sales_df.loc[index, 'Google Address'] = response['results'][0]['formatted_address']
        short_msp_home_sales_df.loc[index, 'Google Lat'] = response['results'][0]['geometry']['location']['lat']
        short_msp_home_sales_df.loc[index, 'Google Long'] = response['results'][0]['geometry']['location']['lng']
    except:
        print(f'Bad Google data for sale ID {index}')
